Consigna:
def userdata(User_id : str ): Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [161]:
import json
import pandas as pd
import gzip
import ast
from datetime import datetime
import numpy as np

In [ ]:
''' Lectura de Archivos '''
# Se abren los 3 dataframes por que el data frame de la función necesita por lo menos un dato de cada uno

df_steam_games = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_steam_games.parquet"
df_steam_games = pd.read_parquet(df_steam_games )
df_steam_games

In [ ]:
df_user_reviews = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_user_reviews.parquet"
df_user_reviews = pd.read_parquet(df_user_reviews)
df_user_reviews

In [ ]:
df_users_items = r"C:\Users\Usuario\Henry\PI1_ML\DataSet-EDAs\df_users_items.parquet"
df_users_items = pd.read_parquet(df_users_items)
df_users_items 

In [ ]:
# Inicio el dataframe de la función por user_items

df_userdata = df_users_items
df_userdata

In [ ]:
df_userdata.info()


In [114]:
del df_users_items

In [167]:
# Cambio el tipo de playtime forever de float 63 a float 32 por espacio

df_userdata['playtime_forever'] = df_userdata['playtime_forever'].astype('float32')

In [ ]:
df_userdata.info()

In [168]:
# De este df solo necesito user_id, item_id, 'items_count', item_name y play time_forever, así que

columnas_borrar = [ 'user_url', 'playtime_2weeks'] 

df_userdata = df_userdata.drop(columnas_borrar,axis=1)

In [ ]:
df_userdata.head()

In [ ]:
# Reviso Stam_Games pues lo relacionare con el precio 
df_steam_games.head()

In [ ]:
# De steam_games solo necesito item_id y price asi que borro las demás columnas
# Item_id = app_name

columnas_borrar2 = ['publisher', 'genres', 'url', 'release_date', 'reviews_url', 'specs', 'id', "early_access", 'developer', 'release_year'] 

df_steam_games = df_steam_games .drop(columnas_borrar2, axis=1)

df_steam_games

In [ ]:
df_steam_games.head()

In [173]:
# Cambio nombre de columna app_name por "item_name" para que pueda hacer el match

df_steam_games = df_steam_games.rename(columns={'app_name': 'item_name'})


In [ ]:
# reviso la columna price de steam-games

df_steam_games.info()

In [175]:
# Modifico de una vez la columna price para que se pueda sumar luego

df_steam_games['price'] = pd.to_numeric(df_steam_games['price'], errors='coerce')
df_steam_games = df_steam_games.dropna(subset=['price'])

In [ ]:
df_steam_games.info()

In [177]:
# Como no me dejó convertir Convierto de float 64 a float 32 por temas de espacio, me toca  crear arreglo y convertirlo luego
#así no me djejo:  df_steam_games['price'] .astype(np.float32)

arreglo = df_steam_games['price'] .to_numpy() 

In [178]:
# convierto el arreglo en float 32

df_float32 = arreglo.astype('float32')

In [179]:
df_float32

array([4.99, 4.99, 4.99, ..., 4.99, 4.99, 4.99], dtype=float32)

In [180]:
# Como ya tengo arreglo en float 32, voy a cambar la columna price por esta en steam_games

df_steam_games['price'] = df_float32


In [ ]:
df_steam_games.info()

In [182]:
# Relaciono steam_games con userdata a travez de item_name para aumentar la coloumna  precio al dataframe

df_userdata = pd.merge(df_userdata, df_steam_games[['item_name', 'price']], on='item_name', how='left')  


In [ ]:
df_userdata.info()

In [133]:
df_userdata.shape

(32746611, 6)

In [184]:
del df_steam_games

In [ ]:
df_userdata.info()

In [136]:
df_userdata = df_userdata.dropna()

In [ ]:
df_userdata.info()

In [186]:
# retiro duplicados

df_userdata = df_userdata.drop_duplicates()


In [ ]:
df_userdata.info()

In [ ]:
df_user_reviews.head()

In [187]:
# solo necesito item_id y recommend  dereviewspor lo tanto borraré las otras columnas

columnas_borrar3 = ['user_id', 'user_url', 'posted', 'sentiment_analysis' ]

In [188]:
df_user_reviews = df_user_reviews.drop(columnas_borrar3, axis=1)

In [ ]:
df_user_reviews.head()

In [ ]:
df_user_reviews.info()

In [191]:
df_user_reviews.shape

(59305, 2)

In [192]:
df_user_reviews = df_user_reviews.dropna()

In [193]:
df_user_reviews = df_user_reviews.drop_duplicates()

In [ ]:
df_user_reviews.head()

In [ ]:
df_userdata.head()

In [198]:
# Solo queda faltando la columna recommend de user reviews y hare la relación a travez de item_id}

df_userdata = pd.merge(df_userdata, df_user_reviews[['item_id', 'recommend']], on='item_id', how='left')  



In [ ]:
df_userdata.head()

In [200]:
# Ahora para la función necesito columna costo que creare multiplicando playtime forever y price y luego hago la columna del porcentaje

df_userdata['cantidad total gastado'] = df_userdata['playtime_forever'] * df_userdata['price']

In [202]:
df_userdata.head()

,user_id,items_count,item_id,item_name,playtime_forever,price,recommend,cantidad total gastado
0,76561197970982479,277,10,Counter-Strike,6.0,9.99,True,59.939999
1,76561197970982479,277,10,Counter-Strike,6.0,9.99,False,59.939999
2,76561197970982479,277,20,Team Fortress Classic,0.0,4.99,False,0.000000
3,76561197970982479,277,20,Team Fortress Classic,0.0,4.99,True,0.000000
4,76561197970982479,277,30,Day of Defeat,7.0,4.99,True,34.930000


In [203]:
# Tambien necesito columna del porcentaje (Hago columna del porcentaje de registros del mismo item con true) para eso primero cantidad de registros por item_id y tiene une excepcion para que no divida por cero

porcentaje_true = (df_userdata['recommend'].mean() * 100).round(2)


In [205]:
df_userdata['porcentaje_true'] = porcentaje_true

In [ ]:
df_userdata.head()
# Claramente la columna funciona

In [207]:
# Borro las columnas que no me sirven o que ya utilice

columnas_borrar4 = ["items_count", "item_name", "playtime_forever", "price", "recommend"]

In [208]:
df_userdata = df_userdata.drop(columnas_borrar4, axis=1)

In [ ]:
df_userdata.info()

In [214]:
df_userdata.to_parquet('df_userdata.parquet')
df_userdata

,user_id,item_id,cantidad total gastado,porcentaje_true
0,76561197970982479,10,59.939999,56.61
1,76561197970982479,10,59.939999,56.61
2,76561197970982479,20,0.000000,56.61
3,76561197970982479,20,0.000000,56.61
4,76561197970982479,30,34.930000,56.61
...,...,...,...,...
8115033,76561198147832762,373330,NaN,56.61
8115034,76561198147832762,373330,NaN,56.61
8115035,76561198147832762,388490,NaN,56.61
8115036,SomeRandomNewAccount,521570,NaN,56.61
